In [478]:
# coding: utf-8
import sys
import re
import string
import ast
from ast import AST
import tokenize
import token
from numbers import Number
import json
import io

In [449]:
def regexEnd(text, start, parentEnd, nodeStart):
    endLineno = None
    endCh = None
    first = text[start]
    m = None
    
    if first in parens:
        # find outer parens
        outer = re.compile("\((.*)\)")
        m = outer.search(text[start:parentEnd+1])
    elif first in sqParens:
        # find outer parens
        outer = re.compile("\[(.*)\]")
        m = outer.search(text[start:parentEnd+1])
    elif first in brackets:
        # find outer parens
        outer = re.compile("\{(.*)\}")
        m = outer.search(text[start:parentEnd+1])
    else: return None
    
    lines = m.group(0).split("\n")
    end = len(m.group(0)) + start
    endLineno = nodeStart['line'] + len(lines)
    endCh = len(lines[-1]) - 1
    if (endLineno == nodeStart['line']): endCh += nodeStart['ch']
    myEnd = {'line': endLineno - 1, 'ch': endCh}
    return (end, myEnd,  m.group(0))

In [523]:
def findNodeStart(node):
    if hasattr(node, 'lineno'):
        return {'line': node.lineno, 'ch': node.col_offset}
    elif  type(node).__name__ == "Module":
        return {'line': 1, 'ch': 0}
    else: # must be some kind of wrapper node
        children = ast.iter_child_nodes(node)
        firstChild = next(children, None)
        if firstChild is None: return None
        return findNodeStart(firstChild)
        
        
def findNodeEnd(node, nextNode, text, start, end, nodeStart):
    endLineno = None
    endCh = None
    
    if(nextNode is None):
        bracketed = regexEnd(text, start, end, nodeStart)
        if(bracketed): return bracketed
        else:
            snippet = text[start:end+1]
            lines = snippet.split("\n")
            ln = nodeStart['line'] + len(lines) - 1
            ch = len(lines[-1]) - 1
            if (ln == nodeStart['line']): ch += nodeStart['ch']
            return end, {'line': ln, 'ch': ch}
    else:
        potentialEnd = {'line': nextNode.lineno, 'ch': nextNode.col_offset} #may need refinement
        snippet = text[start:end+1]
        lines = snippet.split("\n")
        ln = potentialEnd['line'] -  nodeStart['line']
        chars = start
        for x in lines[:ln+1]:
            chars += len(x) + 1 #add 1 for the \n
        chars -= 1 # remove the last \n
        return chars, potentialEnd
        


In [524]:
def findLiteralEnd(snippet, nodeStart): # what to do about multiline literal???
    bracketed = regexEnd(snippet, 0, len(snippet), nodeStart)
    if(bracketed):
        i, nodeEnd, punct = bracketed
        literal = [{'syntok': p} for p in list(punct)]
    else:
        end = 0
        for i, character in enumerate(snippet):
            if character in punctuation:
                end = i - 1
                break
        literal = snippet[0:end+1]
        ch = len(literal) + nodeStart['ch']
        line = nodeStart['line']
        nodeEnd = {'line': line, 'ch': ch}
        
    print("literal |"+str(literal)+"|")
    return (i, nodeEnd, literal)

In [525]:
def visitLiteral(node, text, start):
    myType = type(node).__name__
    myStart = {'line': node.lineno, 'ch': node.col_offset}
    end, myEnd, myLiteral = findLiteralEnd(text[start:], myStart)
    end += start
    if(isinstance(myLiteral, str)):
        return (end, {'type': myType, 'start': myStart, 'end': myEnd, 'literal': myLiteral})
    else: # actually a list of syntok
        return (end, {'type': myType, 'start': myStart, 'end': myEnd, 'content': myLiteral})

In [526]:
def findNextChild(children, itr):
    banned = ["Store", "Load"]
    if(itr + 1 < len(children)):
        child = children[itr + 1]
        if(type(child).__name__ not in banned):
            return child, itr + 1
        else:
            return findNextChild(children, itr + 1)
    else:
        return None, itr + 1

In [527]:
def captureComment(text, textStart, textEnd):
    line = text[textStart:textEnd]
    line = line[:line.find("\n")]
    return textStart + len(line), line

In [528]:
def visit(node, text, textStart, textEnd, nextNode):
    # 1. first, figure out if we're dealing with a literal or parent
    # 2. figure out the start and end positions of this node
    # 3. process children
    # 4. create the formatted Nodey
    
    children = list(ast.iter_child_nodes(node))
    child_itr = -1
    child, child_itr = findNextChild(children, child_itr)
    print("\n",type(node).__name__, list(ast.iter_child_nodes(node)))
    print(child)
    
    if not child: # LITERAL
        return visitLiteral(node, text, textStart)
    
    myType = type(node).__name__
    myContent = [] 
    myStart = findNodeStart(node)
    textEnd, myEnd = findNodeEnd(node, nextNode, text, textStart, textEnd, myStart)
    
    

    # now give each child a span of the text
    nextChild, child_itr = findNextChild(children, child_itr)
    i = textStart
    lineno = myStart['line']
    col_offset = myStart['ch']
    print("text?", textEnd, textStart, lineno, col_offset)
    while i < textEnd:
        token = text[i]
        if child: print(child, lineno, col_offset, child.lineno, child.col_offset, token)
        if child and lineno == child.lineno and col_offset == child.col_offset:
            print(myContent)
            new_i, childContent = visit(child, text, i, textEnd, nextChild)
            myContent.append(childContent)
            col_offset += (new_i - i) - 1
            i = new_i - 1
            print(myType," AFTER: ",myContent, i, "\n")
            child = nextChild
            nextChild, child_itr = findNextChild(children, child_itr)
        elif token == "#":
            new_i, comment = captureComment(text, i, textEnd)
            myContent.append({'syntok': comment})
            print("COMMENT?", comment, new_i, "|"+text[new_i]+"|")
            i = new_i - 1
        elif token in newline:
            lineno += 1
            col_offset = -1
            if not child: textEnd = i #end of this node actually
        else:
            myContent.append({'syntok':token})
        i += 1
        col_offset += 1
    
    me = {'type': myType, 'start': myStart, 'end': myEnd, 'content': myContent}
    return textEnd, me
        
    
    

In [529]:
def parse(text):
    node = ast.parse(text)
    print(ast.dump(node, False, True))
    print("\n",visit(node, text, 0, len(text), None))

In [530]:
text = """# TODO: Select three indices of your choice you wish to sample from the dataset
indices = []

# Create a DataFrame of the chosen samples
samples = pd.DataFrame(data.loc[indices], columns = data.keys()).reset_index(drop = True)
print("Chosen samples of wholesale customers dataset:")
display(samples)"""


sqParens = set(["[","]"])
parens = set(["(",")"])
brackets = set(["{","}"])
newline = set(["\n"]) #todo may vary across platforms
punctuation = set(string.punctuation)
punctuation.add(" ")


# to hurry up, reduce ast at this stage?
# match parens [] {} () otherwise those can end up in weird places

parse(text)
#print(json.dumps(main(l, tree),  indent=2))

Module([Assign([Name('indices', Store(), lineno=2, col_offset=0)], List([], Load(), lineno=2, col_offset=10), lineno=2, col_offset=0), Assign([Name('samples', Store(), lineno=5, col_offset=0)], Call(Attribute(Call(Attribute(Name('pd', Load(), lineno=5, col_offset=10), 'DataFrame', Load(), lineno=5, col_offset=10), [Subscript(Attribute(Name('data', Load(), lineno=5, col_offset=23), 'loc', Load(), lineno=5, col_offset=23), Index(Name('indices', Load(), lineno=5, col_offset=32)), Load(), lineno=5, col_offset=23)], [keyword('columns', Call(Attribute(Name('data', Load(), lineno=5, col_offset=52), 'keys', Load(), lineno=5, col_offset=52), [], [], lineno=5, col_offset=52))], lineno=5, col_offset=10), 'reset_index', Load(), lineno=5, col_offset=10), [], [keyword('drop', NameConstant(True, lineno=5, col_offset=84))], lineno=5, col_offset=10), lineno=5, col_offset=0), Expr(Call(Name('print', Load(), lineno=6, col_offset=0), [Str('Chosen samples of wholesale customers dataset:', lineno=6, col_off

AttributeError: 'keyword' object has no attribute 'lineno'